In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

import chardet
from bs4 import UnicodeDammit

import re

# Считывание данных в формате .csv в DataFrame

In [67]:
data_encodings = []

with open("Data\credit_train.csv", 'rb') as f:
    rawdata = b''.join([f.readline() for _ in range(1000)])
    data_encodings.append(chardet.detect(rawdata)['encoding'])
    data_encodings.append(UnicodeDammit(rawdata).original_encoding)
       
print(f"Кодировка исходного файла: {data_encodings}")

Кодировка исходного файла: ['KOI8-R', 'koi8-r']


Однако, кодировка KOI8-R не подходит для корректного открытия файла, поэтому было решено проверить наиболее распространенные кодировки для файлов, содержащих кириллицу.

In [69]:
data_encodings.extend(['utf-16','CP866', 'Windows-1251', 'utf-8', 'cp1251'])

In [72]:
file_encoding = ''
for encoding in data_encodings:
    try:
        with pd.read_csv("Data\credit_train.csv", encoding=encoding, sep=";", index_col='client_id', iterator=True) as reader:
            print('--------------------------------')
            print(f'Кодировка: {encoding}')
            print(reader.get_chunk(5))
            print('--------------------------------\n')
        if input("Если кодировка верна, введи 'верно' >>>>  ") == 'верно':
            file_encoding = encoding
    except UnicodeDecodeError:
        print(f'Кодировка: {encoding} не подходит!')

--------------------------------
Кодировка: KOI8-R
          gender   age marital_status job_position credit_sum  credit_month  \
client_id                                                                     
1              M   NaN            NaN          UMN   59998,00            10   
2              F   NaN            MAR          UMN   10889,00             6   
3              M  32.0            MAR          SPC   10728,00            12   
4              F  27.0            NaN          SPC   12009,09            12   
5              M  45.0            NaN          SPC        NaN            10   

           tariff_id score_shk education        living_region  monthly_income  \
client_id                                                                       
1                1.6       NaN       GRD   йпюямндюпяйхи йпюи         30000.0   
2                1.1       NaN       NaN               лняйбю             NaN   
3                1.1       NaN       NaN      нак яюпюрнбяйюъ          

Если кодировка верна, введи 'верно'верно


In [195]:
init_data = pd.read_csv("Data\credit_train.csv", encoding=file_encoding, sep=";")
init_data.head(5)

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


In [196]:
print(f'Число наблюдений - {init_data.shape[0]}')
print(f'Число фичей (переменных) - {init_data.shape[1]}')

Число наблюдений - 170746
Число фичей (переменных) - 15


# Предварительная подготовка данных

## Удаление очевидных бесполезных переменных

В данном случае имеются ввиду переменные, у которых количество категорий совпадает с количеством наблюдений, или, наоборот, переменные с одним уникальным значением.

In [197]:
init_data.nunique()

client_id               170746
gender                       2
age                         54
marital_status               5
job_position                18
credit_sum               42769
credit_month                31
tariff_id                   32
score_shk                16279
education                    5
living_region              301
monthly_income            1591
credit_count                21
overdue_credit_count         4
open_account_flg             2
dtype: int64

Откинем столбец `client_id` поскольку он совпадает с номером наблюдения и не несет никакой предсказательной способности.

In [198]:
init_data.drop('client_id', inplace=True, axis=1)

In [199]:
init_data.head(5)

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


## Преобразование типов данных

В данном разделе подразумевается преобразование типов данных к корректным, например, `credit_sum` в датасете имеет тип `object`, чего не должно быть. Также, часто переменные-флаги, такие как "наличие оттока", "наличие дефолта" и т.п. записываются как целочисленные значения.

In [200]:
init_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  object 
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  float64
 7   score_shk             170739 non-null  object 
 8   education             170741 non-null  object 
 9   living_region         170554 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 18.

In [201]:
for column in ['score_shk', 'credit_sum']:
    init_data[column] = init_data[column].str.replace(',', '.').astype('float')

In [202]:
for column in ['tariff_id', 'open_account_flg']:
    init_data[column] = init_data[column].astype('object')

In [203]:
init_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  float64
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  object 
 7   score_shk             170739 non-null  float64
 8   education             170741 non-null  object 
 9   living_region         170554 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  object 
dtypes: float64(6), int64(1), object(7)
memory usage: 18.

In [204]:
init_data.head(5)

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,59998.00,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,10889.00,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,10728.00,12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,12009.09,12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


## Нормализация строковых значений

В данный этап обработки данных входит удаление лишних символов, приведение строк к одному и тому же регистру.
В исследуемом датасете необходимо обработать колонку `living_region`, поскольку уже по нескольким первым записям видно, что вид регионов не стандартизован.

In [351]:
init_data['living_region'].nunique()

84

In [334]:
pd.options.display.max_rows = 310

In [ ]:
init_data['living_region'].value_counts(dropna=False).sort_index()

In [335]:
region_unique = init_data['living_region'].unique()
regions = pd.Series(data=region_unique, index=region_unique, name='regions')

In [336]:
region_stopwords = ['ОБЛ','ОБЛАСТЬ', 'РЕСП', 'РЕСПУБЛИКА', 'КРАЙ', 'Г', 'АО', 'АОБЛ', 'АВТОНОМНАЯ']

In [337]:
def clean_region(region : str, stopwords : list) -> str:
    """
    Deletes stopwords in the region string gained from the stopword-list.
    Returns the remained words joined without spaces.
    
    Keyword arguments:
    region -- name string of the region
    stopwords -- the list of words to be deleted from the region name
    """
    words = re.sub('[,.]+', ' ', str(region)).split(' ')
    cleaned_name = ''
    for word in words:
        if word not in stopwords:
            cleaned_name += word
    return cleaned_name

In [338]:
regions_normalized = regions.apply(clean_region, args=(region_stopwords,))

In [339]:
regions_normalized['ЧУКОТСКИЙ АO'] = 'ЧУКОТСКИЙ'
regions_normalized['ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ'] = 'ЧУВАШСКАЯ'
regions_normalized['ЧУВАШИЯ ЧУВАШСКАЯ РЕСПУБЛИКА -'] = 'ЧУВАШСКАЯ'
regions_normalized['ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП'] = 'ЧУВАШСКАЯ'
regions_normalized['РЕСП ЧУВАШСКАЯ - ЧУВАШИЯ'] = 'ЧУВАШСКАЯ'
regions_normalized['ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП'] = 'ЧУВАШСКАЯ'
regions_normalized['РЕСПУБЛИКАТАТАРСТАН'] = 'ТАТАРСТАН'
regions_normalized['ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ'] = 'МОСКОВСКАЯ'
regions_normalized['ПЕРМСКАЯ ОБЛ'] = 'ПЕРМСКИЙ'
regions_normalized['ОРЁЛ'] = 'ОРЛОВСКАЯ'
regions_normalized['Г.ОДИНЦОВО МОСКОВСКАЯ ОБЛ'] = 'МОСКОВСКАЯ'
regions_normalized['МЫТИЩИНСКИЙ Р-Н'] = 'МОСКОВСКАЯ'
regions_normalized['МОСКОВСКИЙ П'] = 'МОСКОВСКАЯ'
regions_normalized['КАМЧАТСКАЯ ОБЛАСТЬ'] = 'КАМЧАТСКИЙ'
regions_normalized['ДАЛЬНИЙ ВОСТОК'] = 'МОСКОВСКАЯ'
regions_normalized['ДАЛЬНИЙВОСТОК'] = 'МОСКОВСКАЯ'
regions_normalized['ГУСЬ-ХРУСТАЛЬНЫЙ Р-Н'] = 'ВЛАДИМИРСКАЯ'
regions_normalized['ГОРЬКОВСКАЯ ОБЛ'] = 'НИЖЕГОРОДСКАЯ'
regions_normalized['ЭВЕНКИЙСКИЙ АО'] = 'КРАСНОЯРСКИЙ'
regions_normalized['ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions_normalized['АО ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions_normalized['АО ХАНТЫ-МАНСИЙСКИЙ-ЮГРА'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions_normalized['СЕВ. ОСЕТИЯ - АЛАНИЯ'] = 'СЕВЕРНАЯОСЕТИЯ-АЛАНИЯ'
regions_normalized['РЕСП. САХА (ЯКУТИЯ)'] ='САХА/ЯКУТИЯ/'
regions_normalized['РЕСПУБЛИКА САХА'] = 'САХА/ЯКУТИЯ/'
regions_normalized['ДАЛЬНИЙВОСТОК'] = 'МОСКОВСКАЯ'
regions_normalized['САХА'] = 'САХА/ЯКУТИЯ/'
regions_normalized['98'] = 'САНКТ-ПЕТЕРБУРГ'
regions_normalized['74'] = 'ЧЕЛЯБИНСКАЯ'
regions_normalized['РОССИЯ'] = 'МОСКОВСКАЯ'
regions_normalized['МОСКВОСКАЯ'] = 'МОСКОВСКАЯ'
regions_normalized['МОСКВОСКАЯ ОБЛ'] = 'МОСКОВСКАЯ'
regions_normalized['ЧЕЛЯБИНСК'] = 'ЧЕЛЯБИНСКАЯ'
regions_normalized['Г. ЧЕЛЯБИНСК'] = 'ЧЕЛЯБИНСКАЯ'
regions_normalized['БРЯНСКИЙ'] = 'БРЯНСКАЯ'
regions_normalized[np.NaN] = np.NaN

In [340]:
regions_normalized.sort_index()

74                                                    ЧЕЛЯБИНСКАЯ
98                                                САНКТ-ПЕТЕРБУРГ
АДЫГЕЯ РЕСП                                                АДЫГЕЯ
АЛТАЙСКИЙ                                               АЛТАЙСКИЙ
АЛТАЙСКИЙ КРАЙ                                          АЛТАЙСКИЙ
АМУРСКАЯ ОБЛ                                             АМУРСКАЯ
АМУРСКАЯ ОБЛАСТЬ                                         АМУРСКАЯ
АО НЕНЕЦКИЙ                                              НЕНЕЦКИЙ
АО ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю         ХАНТЫ-МАНСИЙСКИЙ
АО ХАНТЫ-МАНСИЙСКИЙ-ЮГРА                         ХАНТЫ-МАНСИЙСКИЙ
АО ЯМАЛО-НЕНЕЦКИЙ                                  ЯМАЛО-НЕНЕЦКИЙ
АОБЛ ЕВРЕЙСКАЯ                                          ЕВРЕЙСКАЯ
АРХАНГЕЛЬСКАЯ                                       АРХАНГЕЛЬСКАЯ
АРХАНГЕЛЬСКАЯ ОБЛ                                   АРХАНГЕЛЬСКАЯ
АРХАНГЕЛЬСКАЯ ОБЛАСТЬ                               АРХАНГЕЛЬСКАЯ
АСТРАХАНСК

In [341]:
init_data['living_region'] = init_data['living_region'].map(regions_normalized)

In [350]:
init_data['living_region'].value_counts(dropna=False).sort_index()

АДЫГЕЯ                     554
АЛТАЙ                       54
АЛТАЙСКИЙ                  553
АМУРСКАЯ                   853
АРХАНГЕЛЬСКАЯ             1723
АСТРАХАНСКАЯ              2719
БАШКОРТОСТАН              6466
БЕЛГОРОДСКАЯ               750
БРЯНСКАЯ                   801
БУРЯТИЯ                   1643
ВЛАДИМИРСКАЯ              1547
ВОЛГОГРАДСКАЯ             2361
ВОЛОГОДСКАЯ               2605
ВОРОНЕЖСКАЯ               1723
ДАГЕСТАН                    69
ЕВРЕЙСКАЯ                  203
ЗАБАЙКАЛЬСКИЙ             1228
ИВАНОВСКАЯ                 944
ИНГУШЕТИЯ                   19
ИРКУТСКАЯ                 4323
КАБАРДИНО-БАЛКАРСКАЯ       637
КАЛИНИНГРАДСКАЯ            728
КАЛМЫКИЯ                   305
КАЛУЖСКАЯ                 1139
КАМЧАТСКИЙ                 412
КАРАЧАЕВО-ЧЕРКЕССКАЯ       576
КАРЕЛИЯ                    679
КЕМЕРОВСКАЯ               2890
КИРОВСКАЯ                  647
КОМИ                      1860
КОСТРОМСКАЯ                514
КРАСНОДАРСКИЙ             8355
КРАСНОЯР

In [352]:
init_data['living_region'].nunique()
# Согласно википедии в России 85 субъектов Российской Федерации

84

In [353]:
init_data.tail(10)

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
170736,F,53.0,MAR,SPC,7769.00,12,1.1,0.346030,SCH,ИРКУТСКАЯ,10500.0,1.0,0.0,0
170737,M,42.0,MAR,SPC,23827.00,10,1.1,0.451455,SCH,КАЛУЖСКАЯ,70000.0,0.0,0.0,0
170738,M,26.0,UNM,SPC,22347.00,10,1.32,0.320288,SCH,САНКТ-ПЕТЕРБУРГ,85000.0,1.0,0.0,0
170739,F,32.0,MAR,SPC,15282.00,10,1.16,0.514811,GRD,САМАРСКАЯ,25000.0,2.0,0.0,0
170740,F,24.0,UNM,SPC,19818.00,12,1.6,0.624391,SCH,КРАСНОДАРСКИЙ,22000.0,1.0,0.0,0
170741,F,27.0,UNM,SPC,64867.00,12,1.1,0.535257,GRD,ТАТАРСТАН,40000.0,6.0,0.0,0
170742,F,24.0,MAR,SPC,17640.00,6,1.6,0.573287,SCH,САНКТ-ПЕТЕРБУРГ,30000.0,1.0,0.0,0
170743,F,31.0,UNM,SPC,27556.47,10,1.32,0.416098,GRD,ПРИМОРСКИЙ,40000.0,1.0,0.0,0
170744,F,53.0,DIV,PNA,6189.00,12,1.1,0.482595,SCH,ПЕНЗЕНСКАЯ,31000.0,2.0,0.0,0
170745,M,49.0,MAR,SPC,12787.00,10,1.1,0.316087,GRD,МОСКОВСКАЯ,40000.0,3.0,0.0,0
